<div style="background-color: #04da79; border: 2px solid #039754; padding: 10px; color: black; font-family: Arial, Helvetica, sans-serif;">
    <strong>Kernel: Python 3 (ipykernel)
</div>

# Fine-tuning Qwen2-VL using the SWIFT framework on video/text dataset

We show how to fine-tune a vision Large Language Model (visionLLM) using the [SWIFT framework]((https://github.com/modelscope/ms-swift)) on a video-text to text dataset. 
The visionLLM can answer questions and extract information from the videos.

A SageMaker remote function is being used to run the GPU intensive based training as a cost efficient SageMaker training job, where you only pay for the actual execution time and resources required by your job. 

## Why SWIFT for fine-tuning?
[SWIFT](https://github.com/modelscope/ms-swift) (Scalable lightWeight Infrastructure for Fine-Tuning) is a comprehensive open-source framework designed to streamline the fine-tuning of large language models (LLMs) and multimodal large language models (MLLMs). 
As the most extensive fine-tuning framework available, SWIFT offers several key advantages:

Key Features:
- Supports over 350+ LLMs and 100+ MLLMs, making it the most comprehensive open-source framework for model fine-tuning
- First training framework to provide systematic support for MLLMs
- Integrates post-training processes including inference, evaluation, and model quantization
- Unified interface for both text and multi-modal model training
- Extensive dataset support with over 150 pure text and multi-modal datasets

Technical Capabilities:
- Compatible with multiple model architectures including Transformer and Mamba
- Supports Megatron structured models for large-scale parallel pre-training
- Implements state-of-the-art tuning techniques that can be used independently
- Provides quantization options (BNB/GPTQ/AWQ) and LoRA merging capabilities
- Enables deployment through PyTorch native and acceleration via vLLM and LMDeploy

SWIFT represents a significant advancement in making LLM training more accessible and efficient, providing a complete technical pipeline that reduces the complexity of understanding and utilizing large models.

## Setup

Let's start by importing necessary packages

In [ ]:
import os
from pathlib import Path
import pandas as pd
import sagemaker
from IPython.display import JSON, Video
from huggingface_hub import snapshot_download

## Dataset for Fine-Tuning

We first need to setup the dataset that we want to do the finetuning on.

* Download a small sample dataset based on [LLaVA-Video-178K](https://huggingface.co/datasets/lmms-lab/LLaVA-Video-178K) into your `./local_data` folder
* Upload the dataset to S3

<div style="background-color: #efff94; border: 2px solid #efff94; padding: 10px; color: black; font-family: Arial, Helvetica, sans-serif;">
    Dataset <a href="https://huggingface.co/datasets/lmms-lab/LLaVA-Video-178K">lmms-lab/LLaVA-Video-178K</a> is released to public under <a href="https://www.apache.org/licenses/LICENSE-2.0">Apache 2.0 License</a>.
</div>

In [ ]:
role = sagemaker.get_execution_role()
region = sagemaker.session.Session().boto_region_name
session = sagemaker.Session()
default_bucket_name = session.default_bucket()
dataset_dir = "./local_data"
dataset_s3_prefix = "myvideotrainingdataset"
dataset_s3_uri = f"s3://{default_bucket_name}/{dataset_s3_prefix}"

In [ ]:
file_path = snapshot_download(
    repo_id="malterei/LLaVA-Video-small-swift",
    repo_type="dataset",
    local_dir=dataset_dir
)
print(f"Downloaded dataset to local filepath: {file_path}")

In [ ]:
dataset_s3_uri = session.upload_data(
    file_path, 
    bucket=default_bucket_name, 
    key_prefix=dataset_s3_prefix,
)
print(f"Uploaded data from local: {file_path} to s3: {dataset_s3_uri}")

### View the dataset format

The formatting of the dataset is the following

* query: user query or request to be answered
* response: expected response
* videos: path to the video to be used in the context
* history: previous conersational history. This can be a multi-turn user/assistant conversations

In [ ]:
train_file = os.path.join(
    dataset_dir, 
    "train.jsonl"
)
df = pd.read_json(train_file, lines=True)
df.head(10)

One row of this dataset described in the train.jsonl file looks as follows

In [ ]:
JSON(df.iloc[0].to_dict(), expanded=True)

Lets look into one of the videos from this dataset

In [ ]:
Video(os.path.join(dataset_dir, df.at[0,"videos"]))

## Run fine-tuning as SageMaker training job

Now we want to run the fine-tuning as a training job on SageMake using the remote decorator function.
The remote decorator allows us to execute any python function very quickly as a remote SageMaker training job.
The inputs and outputs will be automatically serialized/deserialized.



Lets define the SageMaker distribution image to be used for us-east-1. 
The URI for other distributions and regions can be found in the [SageMaker Distribution documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-available-images.html#notebooks-available-images-arn).

Here are a few example distributions from the link above:
* **us-east-1:** 885854791233.dkr.ecr.us-east-1.amazonaws.com/sagemaker-distribution-prod:2.1.0-gpu
* **us-west-2:** 542918446943.dkr.ecr.us-west-2.amazonaws.com/sagemaker-distribution-prod:2.1.0-gpu


In [ ]:
import json
import os
import subprocess
from sagemaker.remote_function import remote

In [ ]:
# lets define the sagemaker distribution to use, we use 1.11 for now
import sagemaker
region = sagemaker.session.Session().boto_region_name

if region=="us-east-1":
    sagemaker_dist_uri = "885854791233.dkr.ecr.us-east-1.amazonaws.com/sagemaker-distribution-prod:2.1.0-gpu"
elif region=="us-west-2":
    sagemaker_dist_uri = "542918446943.dkr.ecr.us-west-2.amazonaws.com/sagemaker-distribution-prod:2.1.0-gpu"
else:
    assert False, "Please make sure to manually set the `sagemaker_dist_uri` uri for your specific AWS region using the provided link from the cell above."

By default, the [Amazon SageMaker Python SDK reads configuration](https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk) values from an admin defined or user specific configuration file. This configuration allows all kind of customizations do be made.
Setting the `SAGEMAKER_USER_CONFIG_OVERRIDE` environment variable below overwrites these defaults.
The main settings you will configure below are
* The container image URI that should run the remote function code.
* Python dependencies to install for the remote training.
* Which files from the local working directory not to upload to the remote code. 

In [ ]:
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd() 

Define dependencies needed for fine-tuning:

In [ ]:
%%writefile ./requirements.txt

git+https://github.com/huggingface/accelerate.git@v1.1.0
ms-swift[llm]@git+https://github.com/modelscope/ms-swift.git@d13c431a0f337f6a04df2ce5310c5666b0c0c1f2
git+https://github.com/huggingface/transformers@21fac7abba2a37fae86106f87fcf9974fd1e3830
pyav
qwen_vl_utils==0.0.8
vllm>=0.6.1
decord
optimum
qwen-vl-utils

Create a new config file to set the configuration for running remotely:

In [ ]:
config_yaml = f"""
SchemaVersion: '1.0'
SageMaker:
  PythonSDK:
    Modules:
      RemoteFunction:
        # role arn is not required if in SageMaker Notebook instance or SageMaker Studio
        # Uncomment the following line and replace with the right execution role if in a local IDE
        # RoleArn: <replace the role arn here>
        S3RootUri: s3://{default_bucket_name}
        ImageUri: {sagemaker_dist_uri}        
        InstanceType: ml.g5.12xlarge
        Dependencies: ./requirements.txt
        IncludeLocalWorkDir: true
        PreExecutionCommands:
        - "pip install packaging"
        CustomFileFilter:
          IgnoreNamePatterns:
          - "local_data/*"
          - "outputs/*"
          - "docker-artifacts/*"
          - "evaluation/*"
          - "sample-media/*"
          - "streamlit-ui/*"
          - "accelerate/*"
          - "container/*"
          - "ms-swift/*"
          - "model/*"
          - "*.ipynb"
          - "__pycache__"

"""

print(config_yaml, file=open('config.yaml', 'w'))
print(config_yaml)

The `@remote` decorator in Amazon SageMaker allows you to execute local Python functions as SageMaker training jobs with minimal code modifications. By annotating a function with @remote, SageMaker transforms the function's code into a training job, enabling it to run on managed infrastructure. This approach simplifies scaling and leverages SageMaker's capabilities without extensive changes to your existing codebase. 

- **Simplified Execution**: Wrap your local machine learning code with the @remote decorator to run it as a SageMaker training job, eliminating the need for manual job setup. 

- **Seamless Integration**: Continue developing in your preferred environment, such as Jupyter notebooks or IDEs, and use the decorator to offload computations to SageMaker's managed infrastructure. 

- **Flexible Configuration**: Customize training job parameters directly within the decorator or through a configuration file, specifying instance types, dependencies, and other settings.

In [ ]:
@remote(
    instance_type="ml.g5.12xlarge", 
    volume_size=200, 
    use_spot_instances=False, 
    job_name_prefix=f"multi-modal-finetune", 
    keep_alive_period_in_seconds=300
)
def fine_tune_video(
    training_data_s3, 
    train_data_path="train.jsonl", 
    validation_data_path="validation.jsonl"
):
    
    from swift.llm.utils import SftArguments
    from swift.llm.sft import llm_sft, get_sft_main

    ## copy the training data from input source to local directory
    dataset_dir = "."
    os.makedirs(dataset_dir, exist_ok=True)
    subprocess.run(['aws', 's3', 'cp', training_data_s3, dataset_dir, '--recursive'])
    train_data_local_path = os.path.join(dataset_dir, train_data_path)
    validation_data_local_path = os.path.join(dataset_dir, validation_data_path)

    # set and run the fine-tuning using ms-swift framework
    
    sft_main = get_sft_main(SftArguments, llm_sft)
    os.environ["NFRAMES"]=json.dumps(24) # can be increased, but will require more memory
    os.environ["MAX_PIXELS"]=json.dumps(100352) # can be increased, but will require more memory
    os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3" # devices to be used
    os.environ["NPROC_PER_NODE"]="4" # we have 4 GPUs on this instance
    os.environ["USE_HF"]="1" # use huggingface
    
    argv = ['--model_type', 'qwen2-vl-2b-instruct',
            '--model_id_or_path', 'Qwen/Qwen2-VL-2B-Instruct', 
            '--sft_type', 'lora', 
            '--output_dir', '/opt/ml/model' ,'--max_length', '2048',
            '--dataset', train_data_local_path, 
            '--val_dataset', validation_data_local_path]
    sft_main(argv)
    return "done"

That's all, now run your training job

In [ ]:
print(f"View your job here: https://{region}.console.aws.amazon.com/sagemaker/home?region={region}#/jobs/")
fine_tune_video(
    dataset_s3_uri
)

Wait for the training job to complete. You can see the logs of the training above until it prints 'done' and the cell finishes executing.

<div style="background-color: #f5aeba; border: 2px solid #610010; padding: 10px; color: black; font-family: Arial, Helvetica, sans-serif;">
    TrainingJob on a <b>ml.g5.12xlarge</b> instance it should take <b>16-20 minutes</b>
</div>

## Download Fine-Tuned Model for Evaluation

In [ ]:
base_prefix = 'multi-modal-finetune-'
models_list_s3 = !aws s3api list-objects-v2 --bucket {default_bucket_name} --prefix {base_prefix} --query "Contents[?contains(Key, 'output/model.tar.gz')]|sort_by(@, &LastModified)[-1].Key" --output text
print(f"found {models_list_s3[0]}")

In [ ]:
model_suffix_s3 = models_list_s3[0]
model_s3_path = os.path.join("s3://", default_bucket_name, model_suffix_s3)
print(f"Fine-tuned Model Adapter: {model_s3_path}")

if not model_s3_path.endswith("model.tar.gz"):
    assert False, "No latest fine-tuning found. Did your fine-tuning finish?"

We copy the model from S3 to our local directory

In [ ]:
model_weights_dir = "./model"
model_destination = f"{model_weights_dir}/{model_suffix_s3}"
model_dest_dir = str(Path(model_destination).parent)

In [ ]:
!aws s3 cp {model_s3_path} {model_destination}
!tar --warning=no-unknown-keyword  -xzvf {model_destination} --directory {model_dest_dir} > /dev/null

Lets have a look what is inside of `model.tar.gz`:

* The checkpoint directory contains the actual adapter
* `adapter_model.safetensors` - contains the actual weights of the adapter

For inference we can either use the adapter together with the original model, or we merge the adapter with the original model.

In [ ]:
! cd {model_dest_dir} && du -ah --max-depth=5

In [ ]:
model_identifier = "qwen2-vl-2b-instruct"

In [ ]:
model_dir = os.path.join(model_dest_dir, model_identifier)

Swift outputs the directory that contains the weights of the best model. When training for a short time there might only be one checkpoint.
If there are multiple checkpoints then the best checkpoint will be the model that performs the best on a specific metric. By default the metric is loss (reference `--predict_with_generate` in the [Swift documentation](https://swift.readthedocs.io/en/latest/Instruction/Command-line-parameters.html#sft-parameters)) and the model with the lowest evaluation loss on the validation dataset is considered the best model.

<div style="background-color: #ffe347; border: 2px solid #610010; padding: 10px; color: black; font-family: Arial, Helvetica, sans-serif;">
    You will learn more about what evaluation loss means in the <b>Evaluation</b> chapter.
</div>

In [ ]:
from utils.helpers import find_latest_version_directory, find_best_model_checkpoint

latest_version = find_latest_version_directory(model_dir)
logging_file = os.path.join(os.getcwd(), model_dir, latest_version, "logging.jsonl")
best_model_checkpoint = find_best_model_checkpoint(logging_file)
if best_model_checkpoint:
    best_model_checkpoint = best_model_checkpoint.replace("/opt/ml/model/","")
    print(f"best model checkpoint: {best_model_checkpoint}")
else:
    print("Best model checkpoint not found. Please search the logs manually to find the path that stores the best model checkpoint.")
          

## Evaluation of Fine-Tuned Model

Let's look at the evaluation of a fine-tuning run. To save, you some time for the other labs we have already prepared the evaluation results and analysis for you below.  

### Explanation Evaluation Metrics from Training

When training a machine learning model it makes sense to split the dataset into a training dataset, validation dataset, and an optional test dataset. 

| Dataset Type | Purpose | Usage Phase | Characteristics | Model Interaction |
|--------------|---------|-------------|-----------------|-------------------|
| Training | Used to teach the model by adjusting its parameters and learning patterns. | Initial Training | Largest subset of data; must be representative of the overall dataset | Model actively learns and adjusts parameters based on this data |
| Validation | Provides unbiased evaluation during training, prevent overfitting, helps fine-tune hyperparameters | During Training | Separate from training data; used for model optimization | Parameters can be adjusted based on performance, select between different model candidates |
| Test | Offers final, unbiased evaluation of model performance | After Training | Completely unseen data; simulates real-world scenarios | No adjustments made; used only for final assessment |


Metrics that give insights into the performance of the training process are accuracy and loss.

#### Accuracy
Accuracy measures how well your model performs on a dataset, expressed as a percentage of correct predictions. 
In this case the default accuracy used for fine-tuning is token level accuracy (parameter `--acc_strategy` in [swift documentation](https://swift.readthedocs.io/en/latest/Instruction/Command-line-parameters.html#sft-parameters)). [Token level accuracy is](https://github.com/modelscope/ms-swift/blob/552190740c1fefa836407a8dff53f5153bfbfa7e/swift/utils/metric.py#L53) defined as the number of correctly predicted tokens divided my the number of total predicted tokens. A token is the smallest unit that a language model generates. It is similar to words in the human vocabulary.

Here is an example:
| Metric | Dataset Label | Model Prediction |
|--------|-------|------------|
| Full Text | A. Hello World! | B. Hello Planet! |
| Token IDs | [32, 13, 21927, 1879, 0] | [33, 13, 21927, 28835, 0] |

_Matches:_ 3/5 tokens correct  
_Accuracy:_ 60%

For this sample the model output was [33, 13, 21927, 28835, 0] which decoded is "B. Hello Planet!". The label in the dataset is "A. Hello World!". So for this sample the accuracy is 60%.

Accuracy helps you understand:
* How well your model has learned the patterns in your data
* Whether your model has sufficient capacity to learn the task
* The overall progress of the learning process

#### Loss
Loss is a more nuanced metric that quantifies the model’s prediction errors during training. It represents the “cost” or “penalty” the model incurs for making incorrect or imprecise predictions. Key aspects include:
The loss function depends on the model architecture. [Qwen2 VL uses Cross Entropy](https://github.com/huggingface/transformers/blob/1a0cd69435cf6ddfef2b15e37cbddd94e13348f4/src/transformers/models/qwen2_vl/modeling_qwen2_vl.py#L30) as the loss function.

Cross-entropy loss measures how well a language model predicts the next token in a sequence by comparing:
* The model's predicted probability distribution over all possible next tokens
* The actual token that appears in the dataset

Loss calculation examples:
* If model is very confident (90%) and correct: Small loss
* If model is unsure (33%): Medium loss
* If model is confident but wrong (5% for correct answer): Large loss

Key Points
* Loss is smallest when the model is confident and correct
* Loss increases when the model:
    * Is uncertain about the correct answer
    * Is confident about the wrong answer
* Lower loss values indicate better model performance
* Perfect zero loss is impossible because language has natural ambiguity
* It provides a more sensitive measure of model improvement than accuracy

### View Evaluation Metrics from actual fine-tuning

Now we apply the accuracy and loss metrics to the training dataset and validation (evaluation) dataset from an actual training run.

The swift library captures images of training metrics in the `images` directory of the model directory.
Let's take a look at the training metrics of a training job with a [dataset of 5000 samples](https://huggingface.co/datasets/malterei/LLaVA-Video-large-swift). Fine-tuning Qwen2 VL 2B on the larger 5000 samples dataset took 5 hours and 39 minutes on a `ml.g5.48xlarge` instance.

#### Accuracy and loss on Training Dataset

In [ ]:
from IPython.display import Image
from IPython.display import display

train_acc = Image(filename='evaluation/qwen2-vl-2b-instruct/large/v0-20241120-161510/images/train_acc.png') 
train_loss = Image(filename='evaluation/qwen2-vl-2b-instruct/large/v0-20241120-161510/images/train_loss.png')

Below is the training accuracy graph (**train/acc**). The model starts with relatively low **accuracy** (60%) and quickly improves within the first 25 epochs, reaching about 90% accuracy. After this initial sharp improvement, the accuracy stabilizes and fluctuates between 93-96% for the remainder of the training. This pattern demonstrates successful learning with diminishing returns over time.

In [ ]:
display(train_acc)

The training loss (**train/loss**) graph below shows a classic exponential decay pattern that is typical of successful model training. 

In [ ]:
display(train_loss)

Together the training accuracy and training loss indicate that:
* The model reaches a stable state without signs of overfitting.
* The smooth curves suggest the learning rate was well-chosen for this task.
* Improvements after the 200th epoch are minimal and early stopping would make sense.

#### Accuracy and loss on Validation Dataset

The evaluation metrics show the performance of the model on the validation dataset. 

The validation ensures that the model does not just memorize the training data but actually learns something about the task from the data.

In [ ]:
eval_acc = Image(filename='evaluation/qwen2-vl-2b-instruct/large/v0-20241120-161510/images/eval_acc.png') 
eval_loss = Image(filename='evaluation/qwen2-vl-2b-instruct/large/v0-20241120-161510/images/eval_loss.png') 

Below are the graphs for the evaluation accuracy (**eval/acc**) and evaluation loss (**eval/loss**).
Both evaluation metrics show consistent improvement without erratic behavior. This gradual improvement suggests good generalization to unseen real-world data.

The evaluation accuracy demonstrates consistent improvement, starting at 96% around epoch 50 and gradually climbing to approximately 97% by epoch 300, with a slight peak around epoch 200. When compared to the training metrics, the relatively small gap between training and evaluation performance (training accuracy ~95% vs evaluation accuracy ~97%) suggests the model is not overfitting. 

Overfitting means the model would only learn the training data and not generalize well. In the metrics this would show if the training metrics keep improving while the evaluation metrics worsen or plateau.

The evaluation loss shows a steady decrease from 0.125 to 0.095 over 300 epochs, with a notably smooth descent curve that indicates stable learning on unseen data. 


The relatively high accuracy and low loss values suggest this is a well-performing model on its intended multiple choice question answering task.

In [ ]:
display(eval_acc)
display(eval_loss)

### Real-World Performance

<div style="background-color: #ffe347; border: 2px solid #610010; padding: 10px; color: black; font-family: Arial, Helvetica, sans-serif;">
    You will not run inference for evaluation in this notebook. To save time and to optimize for maximal learning we already ran inference with different models for comparison. 
Please use the optional <a href="./04-02_optional_fine_tune_video_inference.ipynb">notebook 04-02_optional_fine_tune_video_inference.ipynb</a> if you want to run inference on the small fine-tuned model yourself.
</div>

In addition to the training and evaluation metrics we have also run inference on an additional test dataset that has not been used during training.


Large Language Models are challenging to evaluate due to their dynamic output generation, compared to simple classification tasks. The evaluation approach depends on the specific task and data, requiring data and task specific quantitative metrics and/or qualitative assessment methods for generated text, where multiple valid outputs may exist.

Our dataset specifically is multiple choice questions on a video. In this case we have a correct multiple choice answer and can use that as a quantitative metric.

Below you can see the accuracy (percentage of correct multiple choice answers) on the test dataset for the pre-trained model out of the box, the decent performance from fine-tuning on a small dataset, and the good performance when fine-tuning on a larger dataset for an extended time.

| Model | Instance | Fine-Tuning Duration | Dataset Size | Accuracy |
|----------|----------|----------|----------|----------|
| Qwen2 VL 2B (vanilla)    | no fine-tuning     | no fine-tuning     | N/A | 50% |
| Qwen2 VL 2B Fine-Tuned Small    | ml.g5.12xlarge     | 24 minutes     | 100 | 50% |
| Qwen2 VL 2B Fine-Tuned Large    | ml.g5.48xlarge     | 5 hours 39 minutes     | 4660 | 83% |

We load the inference results from the three models:

In [ ]:
qwen2_2b_responses = pd.read_json(
    "evaluation/qwen2-vl-2b-instruct/vanilla/outputs.jsonl",
    lines=True
)
qwen2_2b_fine_tuned_responses_small = pd.read_json(
    "evaluation/qwen2-vl-2b-instruct/small/outputs.jsonl",
    lines=True
)
qwen2_2b_fine_tuned_responses = pd.read_json(
    "evaluation/qwen2-vl-2b-instruct/large/outputs.jsonl",
    lines=True
)
evaluation_responses  = pd.merge(
    qwen2_2b_responses, 
    qwen2_2b_fine_tuned_responses_small[['query','response', 'response_choice']],
    on="query",
    suffixes=('', '_small_ft_qwen2_vl_2b')
)
evaluation_responses  = pd.merge(
    evaluation_responses, 
    qwen2_2b_fine_tuned_responses[['query','response', 'response_choice']],
    suffixes=('_vanilla_qwen2_vl_2b','_large_ft_qwen2_vl_2b'),
    on="query"
)

Below we render the input and outputs from the three different models:
1. out of the box pre-trained vanilla Qwen2 VL 2B model
2. fine-tuned Qwen2 VL 2B model on a small dataset
3. fine-tuned Qwen2 VL 2B model on a large dataset

We only show the rows in which one model made an incorrect predictions. Those predictions give us an idea what the model is getting wrong and how the three models are different. Fewer incorrect predictions is better.

The first column in the table below contains the input prompt that is sent to the model along with the video in the second column. The *pre-trained response* column contains the prediction from Qwen2 VL 2B out of the box, the *small fine-tuned response* shows the prediction of a small 24 minutes fine-tuning and the *large fine-tuned response* of Qwen2 VL 2B fine-tuned on the larger dataset for 5 and a half hours. The last column contains the ground truth, so what we expect as the correct answer.

In [ ]:
import re
df = evaluation_responses.copy()

incorrect = (df['label_choice'] != df['response_choice_vanilla_qwen2_vl_2b']) | (df['label_choice'] != df['response_choice_small_ft_qwen2_vl_2b']) | (df['label_choice'] != df['response_choice_large_ft_qwen2_vl_2b'])
df = df[incorrect].reset_index(drop=True)

def format_video(video_url):
    if video_url:
        return f'''<video width="320" height="240" controls>
  <source src="{os.path.join(dataset_dir,video_url)}" type="video/mp4">
Your browser does not support the video tag.
</video>
'''

    return ""

def format_query_options(query_text):
    # First split the question from options
    query_text = query_text.replace("<video>","")
    parts = query_text.split('A.')
    if len(parts) < 2:
        return query_text
        
    question = parts[0].strip()
    options_text = 'A.' + parts[1]
    
    # Format options with line breaks
    formatted_options = re.sub(
        r'([A-D]\. .*?)(?=[A-D]\.|$)', 
        r'\1\n', 
        options_text
    ).strip()
    
    return f"{question}\n{formatted_options}"

# Apply to dataframe
df['videos'] = df['videos'].apply(format_video)
df['query'] = df['query'].apply(format_query_options)


# Select relevant columns for display
display_columns = [
    'query', 
    'videos',
    'response_vanilla_qwen2_vl_2b', 
    'response_small_ft_qwen2_vl_2b',
    'response_large_ft_qwen2_vl_2b',
    'label'
]

def style_cell(is_correct):
    color = '#c8e6c9' if is_correct else '#ffcdd2'  # Light green or light red
    return f'background-color: {color}'

def style_incorrect(column):
    return [style_cell(df.loc[i, column.name.replace("response_","response_choice_")] == df.loc[i, 'label_choice'])
        if re.match('^response_(?!choice)', column.name)
        else ''
        for i, _ in enumerate(column)
    ]

mapper = {
    'label': 'Ground truth',
    'videos': 'Video',
    'query': 'Input prompt',
    'response_vanilla_qwen2_vl_2b':'Pre-trained response', 
    'response_small_ft_qwen2_vl_2b':'Small fine-tuned response',
    'response_large_ft_qwen2_vl_2b':'Large fine-tuned response'
}

styled_df = (df[display_columns].style
    .apply(style_incorrect)
    .set_properties(**{
        'text-align': 'left',
        'padding': '10px',
        'border': '1px solid #ddd',
        'white-space': 'pre-wrap',  # Add this line to preserve newlines
        'height': 'auto',        # Allow height to adjust to content
    })
    .set_table_styles([
        {'selector': 'table', 'props': [
            ('table-layout', 'auto'),  # Allow different row heights
            ('border-collapse', 'collapse')
        ]},
        {'selector': 'th', 'props': [
            ('background-color', '#f5f5f5'),
            ('text-align', 'left'),
            ('padding', '10px'),
            ('border', '1px solid #ddd')
        ]}
    ])
    .set_properties(**{
        'min-width': '220px'
    }, subset=['query'])
    .format_index(mapper.get, axis=1)
)



display(styled_df)

The accuracy of the out of the box model and the small fine-tuned model is the same (50% accuracy). However, in the actual responses you can already see the small fine-tuning taught the model the expected output format of responding with the letter from the available choices (for example "A."). The answer options that the fine-tuned model selects are still 50% incorrect. The hypothesis is that to understand the videos better we will need to fine-tune on more data.

The *large fine-tuned response* column shows that the fine-tuned model on the larger dataset for longer resulted in only two incorrect predictions. We can see that now the model has learned not just the response format but also understanding videos it has never seen and answering multiple-choice questions with good success.

The evaluation showed us that fine-tuning helps to improve the performance of a vision large language model on our task.

For your specific use case we also recommend running a qualitative human analysis of sample or failure cases, to ensure that the metric is aligning with your task specific expectations.

<div style="background-color: #ffe347; border: 2px solid #610010; padding: 10px; color: black; font-family: Arial, Helvetica, sans-serif;">
    Please restart the notebook kernel to free up memory.
</div>

In [ ]:
from IPython import get_ipython
get_ipython().kernel.do_shutdown(restart=True)